In [1]:
import torch
import ipywidgets
import pandas as pd
from pathlib import Path
from omegaconf import OmegaConf
from tqdm.auto import tqdm

In [2]:
%load_ext autoreload
%autoreload 2
import nncomp
import nncomp_molecule

In [3]:
dataset = nncomp_molecule.datasets.ImageCaptioningDataset(
    pd.read_pickle("/work/input/kfujikawa/bms-molecular-translation-preprocess-v1/train.debug.pkl"),
    image_transforms={
        "class": "AlbumentationCompose",
        "transforms": [
            {
                "class": "Resize",
                "height": 224,
                "width": 224
            }
        ]
    },
    inchi_transforms={
        "class": "HFTokenizer",
        "config": "/work/input/kfujikawa/bms-molecular-translation-tokenizers/inchi-atoms-numbers.json",
        "out_columns": ["token_ids", "next_token_ids"],
    },
)

In [4]:
with torch.autograd.profiler.profile(use_cuda=False) as prof:
    dataset[0]
print(prof.key_averages().table(sort_by="self_cpu_time_total"))

--------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
--------------------  ------------  ------------  ------------  ------------  ------------  ------------  
    image_transforms        78.42%       2.424ms        78.45%       2.425ms       2.425ms             1  
    inchi_transforms        18.45%     570.276us        18.52%     572.338us     572.338us             1  
         aten::zeros         2.15%      66.374us         3.03%      93.747us      46.874us             2  
         aten::empty         0.77%      23.683us         0.77%      23.683us       5.921us             4  
         aten::zero_         0.22%       6.690us         0.22%       6.690us       3.345us             2  
--------------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 3.091ms

